# Proof-of-Concept: Attention Visualization for NLI Model

## 1. Load Dependencies

In [ ]:
import sys
sys.path.append('../src') # Add src directory to path

import torch
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification
import numpy as np # Retained for general use, though direct use might be reduced
# matplotlib and seaborn are used by xai_utils.plot_heatmap

from xai_utils import process_model_attentions, plot_heatmap

## 2. Load Fine-tuned Model and Tokenizer

In [ ]:
model_path = '../src/nli_model/' # Relative to notebook location
tokenizer = DistilBertTokenizerFast.from_pretrained(model_path)
model = DistilBertForSequenceClassification.from_pretrained(model_path, output_attentions=True)
model.eval() # Set the model to evaluation mode

## 3. Prepare Sample Input

In [ ]:
premise = "A man is playing a guitar."
hypothesis = "A person is making music."
inputs = tokenizer(premise, hypothesis, return_tensors='pt', truncation=True, padding=True)

## 4. Get Model Prediction and Attentions

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
inputs_on_device = {k: v.to(device) for k, v in inputs.items()} # Ensure inputs are on device

with torch.no_grad():
    outputs = model(**inputs_on_device)

model_attentions_output = outputs.attentions  # Tuple of attention tensors
predicted_class_idx = torch.argmax(outputs.logits, dim=1).item()

label_map = {0: 'entailment', 1: 'contradiction', 2: 'neutral'}
print(f"Premise: {premise}")
print(f"Hypothesis: {hypothesis}")
print(f"Predicted Label: {label_map[predicted_class_idx]} (Class index: {predicted_class_idx})")

## 5. Process Attention Weights using Utility Function

In [ ]:
# Use utility function to process attentions
# inputs['input_ids'] is already on the device from the previous cell, 
# but process_model_attentions handles moving it to CPU for tokenization.
avg_attentions_np, tokens = process_model_attentions(model_attentions_output, inputs['input_ids'], tokenizer)

## 6. Visualize Full Attention (Premise & Hypothesis) using Utility Function

In [ ]:
plot_heatmap(avg_attentions_np, tokens, tokens, title='Full Attention Heatmap - Last Layer (Averaged Heads)')

## 7. Detailed Interpretation of Example Heatmap & Further Attention Analysis

The heatmap above visualizes the token-to-token attention mechanism from the last layer of the DistilBERT model, averaged across all attention heads for the input: 
**Premise:** "A man is playing a guitar."
**Hypothesis:** "A person is making music."

**Qualitative Interpretation:**
- **High Self-Attention on Special Tokens:** The `[CLS]` and `[SEP]` tokens often show high attention to themselves and sometimes to other tokens. The `[CLS]` token's representation is used for classification, so it aggregates information from the entire sequence. `[SEP]` tokens mark boundaries and might also aggregate segment-specific information.
- **Premise-Hypothesis Interaction:** We can observe how tokens in the premise attend to tokens in the hypothesis and vice-versa. For instance, 'guitar' in the premise likely attends strongly to 'music' in the hypothesis, and 'man' to 'person'. These cross-attentions are crucial for the model to determine the relationship (entailment, contradiction, neutral).
- **Within-Sentence Attention:** Tokens within the premise (e.g., 'man', 'playing', 'guitar') will attend to each other to form a contextual understanding of the premise. Similarly, tokens within the hypothesis ('person', 'making', 'music') will attend to each other.
- **Diagonal Dominance (Often):** Often, tokens attend most strongly to themselves or adjacent tokens, which is natural as a token's own identity and immediate context are usually very important. This might be visible as a brighter diagonal line.
- **Sparsity:** Attention is often sparse; a token only attends strongly to a few other tokens. The average across heads might smooth this out, making individual head examination useful.

**Further Attention Analysis Possibilities:**
- **Individual Attention Heads:** Instead of averaging, visualizing attention maps for each head separately can reveal specialized roles. Some heads might focus on syntactic relationships, others on semantic similarity, or specific positional information.
- **Different Layers:** Attention patterns can vary significantly across layers. Early layers might capture more local, syntactic information, while later layers (like the one visualized) might capture more abstract, semantic relationships and task-specific information.
- **Attention Flow:** For more complex models (e.g., full BERT or other architectures), techniques exist to trace how attention 'flows' through the network, aggregating information from input to output. This is less common for DistilBERT due to its distilled nature but is a general concept in attention analysis.
- **Attention to `[CLS]`:** Examining what tokens the `[CLS]` token attends to can highlight which parts of the input sequence are most influential for the final classification decision.

## 8. Separated Attention Views (Premise-Only and Hypothesis-Only) using Utility Function

In [ ]:
sep_idx = -1
try:
    sep_idx = tokens.index(tokenizer.sep_token)
except ValueError:
    print(f"'{tokenizer.sep_token}' not found in tokens. This should not happen for NLI tasks.")

if sep_idx != -1 and len(tokens) > 1:
    premise_tokens = tokens[1:sep_idx]
    # Ensure avg_attentions_np is used here, which is the numpy array from process_model_attentions
    premise_avg_attentions_np = avg_attentions_np[1:sep_idx, 1:sep_idx]
    if premise_tokens and premise_avg_attentions_np.size > 0:
        plot_heatmap(premise_avg_attentions_np, premise_tokens, premise_tokens, title='Premise Self-Attention - Last Layer (Averaged Heads)')
    else:
        print("Not enough tokens to display premise self-attention.")

    hypothesis_tokens = tokens[sep_idx+1:-1]
    hypothesis_avg_attentions_np = avg_attentions_np[sep_idx+1:-1, sep_idx+1:-1]
    if hypothesis_tokens and hypothesis_avg_attentions_np.size > 0:
        plot_heatmap(hypothesis_avg_attentions_np, hypothesis_tokens, hypothesis_tokens, title='Hypothesis Self-Attention - Last Layer (Averaged Heads)')
    else:
        print("Not enough tokens to display hypothesis self-attention.")
else:
    if sep_idx == -1:
        print("Skipping separated attention view plots as SEP token was not found.")
    else:
        print("Skipping separated attention view plots due to insufficient tokens.")

The separated views above show the self-attention within the premise and hypothesis independently. This can help isolate how the model builds contextual understanding for each part of the input before (or while) considering their relationship.

## 9. Further Analysis of Model Internals (Conceptual for PoC)

Beyond attention visualization, a comprehensive XAI analysis for NLI models could include:

- **Word Embeddings Analysis:**
  - *What it is:* Examining the vector representations of words/tokens from the model's embedding layer.
  - *What it might reveal:* Semantic similarities between words, how context influences representations (for contextual embeddings), and potential biases learned by the model. Techniques like PCA or t-SNE can be used for visualization.

- **Layer Activations and TCAV (Testing with Concept Activation Vectors):**
  - *What it is:* Analyzing the output of intermediate layers (activations) to understand what features or concepts these layers respond to. TCAV helps quantify the model's sensitivity to human-understandable concepts.
  - *What it might reveal:* Which parts of the network are responsible for identifying specific linguistic phenomena (e.g., negation, sentiment, syntactic structures) or high-level concepts relevant to NLI.

- **Neuron Activations / Feature Visualization:**
  - *What it is:* Identifying specific neurons in dense layers that activate strongly for particular inputs or classes.
  - *What it might reveal:* What specific features or combinations of features individual neurons have learned to detect.

- **Weight Norms and Pruning Analysis:**
  - *What it is:* Examining the magnitudes of weights in the network. Pruning involves removing less important weights/neurons.
  - *What it might reveal:* Which neurons or connections are most influential for the model's predictions, potentially leading to model compression or identifying critical components.

These methods, often in combination, provide a deeper understanding of the model's decision-making process beyond just input-output behavior.